In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Define paths to your datasets
negative_file_path = "/content/drive/My Drive/zeroes.txt"  # Update with your path
positive_file_path = "/content/drive/My Drive/ones[1].txt"    # Update with your path

# Step 3: Read and preprocess the datasets
def preprocess_sentence(sentence):
    parts = sentence.split(maxsplit=1)
    return parts[-1].strip() if len(parts) > 1 else ""

# Load negative (incorrect) sentences
with open(negative_file_path, 'r', encoding='utf-8') as f:
    negative_sentences = [preprocess_sentence(line) for line in f.readlines() if line.strip()]

# Load positive (correct) sentences
with open(positive_file_path, 'r', encoding='utf-8') as f:
    positive_sentences = [preprocess_sentence(line) for line in f.readlines() if line.strip()]

# Step 4: Combine and label the datasets
data = []
for sentence in negative_sentences:
    data.append({"text": sentence, "label": 0})  # Label 0 for incorrect
for sentence in positive_sentences:
    data.append({"text": sentence, "label": 1})  # Label 1 for correct

# Step 5: Save the combined dataset as a CSV file
import pandas as pd
dataset = pd.DataFrame(data)
output_path = "/content/drive/My Drive/combined_dataset.csv"  # Update with your desired output path
dataset.to_csv(output_path, index=False)

print(f"Combined dataset saved to: {output_path}")


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload the correct_sentences.txt file


Saving sentences.txt to sentences (1).txt


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(file_name, header=None, names=['sentences.txt'])

df.to_csv('sentences.csv', index=False)

NameError: name 'file_name' is not defined

In [ ]:
# Install necessary libraries
!pip install transformers datasets

# File upload
from google.colab import files
uploaded = files.upload()  # Upload correct_sentences.txt

# Load correct sentences
correct_sentences = []
with open('sentences.txt', 'r', encoding='utf-8') as file:
    correct_sentences = file.readlines()

# Generate incorrect sentences
import pandas as pd
import random

def introduce_errors(sentence):
    words = sentence.split()
    num_errors = random.randint(1, max(1, len(words) // 3))  # Introduce 1 to len(words)//3 errors
    for _ in range(num_errors):
        word_idx = random.randint(0, len(words) - 1)
        word = words[word_idx]
        if random.random() < 0.5:
            if len(word) > 1:
                char_idx = random.randint(0, len(word) - 2)
                word = word[:char_idx] + word[char_idx + 1] + word[char_idx] + word[char_idx + 2:]
        else:
            if len(word) > 1:
                char_idx = random.randint(0, len(word) - 1)
                word = word[:char_idx] + word[char_idx + 1:]
        words[word_idx] = word
    return ' '.join(words)

data = {'incorrect': [], 'correct': []}
for correct_sentence in correct_sentences:
    correct_sentence = correct_sentence.strip()
    incorrect_sentence = introduce_errors(correct_sentence)
    data['incorrect'].append(incorrect_sentence)
    data['correct'].append(correct_sentence)

df = pd.DataFrame(data)
df.to_csv('/content/sinhala_sentences.csv', index=False)

# Training
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

dataset = Dataset.from_pandas(df)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def preprocess_function(examples):
    inputs = [ex for ex in examples['incorrect']]
    targets = [ex for ex in examples['correct']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length').input_ids
    model_inputs['labels'] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)
model = T5ForConditionalGeneration.from_pretrained('t5-small')

training_args = TrainingArguments(
    output_dir='/content/results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

trainer.train()

# Save the model
model.save_pretrained('/content/sinhala-grammar-corrector')
tokenizer.save_pretrained('/content/sinhala-grammar-corrector')

# Load and Test
model = T5ForConditionalGeneration.from_pretrained('/content/sinhala-grammar-corrector')
tokenizer = T5Tokenizer.from_pretrained('/content/sinhala-grammar-corrector')

def correct_text(input_text):
    inputs = tokenizer.encode("correct: " + input_text, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

input_text = "මම විදුලී සංගීටය අගනාගයි"
corrected_text = correct_text(input_text)
print("Corrected Text:", corrected_text)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/16920 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.005900,0.003494
2,0.003200,0.001896
3,0.002500,0.001404


Corrected Text: 


In [ ]:
input_text = "උත්සාහශීලී ළමයෙකු විශාල වන අසල ජීවිතය ගත කළා"  # Intentional errors for testing
corrected_text = correct_text(input_text)
print("Corrected Text:", corrected_text)


Corrected Text: 


In [ ]:
# Define input text
input_text = "මම විදුලී සංගීටය අගනාගයි"  # Intentional errors for testing
print("Input Text:", input_text)

# Encode the input text
inputs = tokenizer.encode("correct: " + input_text, return_tensors="pt", max_length=128, truncation=True)
print("Encoded Inputs:", inputs)

# Generate output from the model
outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
print("Model Outputs:", outputs)

# Decode the output
corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Corrected Text:", corrected_text)


Input Text: මම විදුලී සංගීටය අගනාගයි
Encoded Inputs: tensor([[2024,   10,    3,    2,    3,    2,    3,    2,    3,    2,    1]])
Model Outputs: tensor([[0, 3, 2, 3, 2, 3, 2, 1]])
Corrected Text: 


In [ ]:
input_text = "මම ගෙදර ගියෙමු."

# Tokenize the text
tokens = tokenizer.tokenize(input_text)
print("Tokens:", tokens)

# Convert tokens to IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

# Encode the text
encoded_inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=128, truncation=True)
print("Encoded Input IDs:", encoded_inputs)

# Decode back to text
decoded_text = tokenizer.decode(encoded_inputs[0], skip_special_tokens=True)
print("Decoded Text:", decoded_text)


Tokens: ['▁', 'මම', '▁', 'විදුලී', '▁', 'සංගීටය', '▁', 'අගනාගයි']
Token IDs: [3, 2, 3, 2, 3, 2, 3, 2]
Encoded Input IDs: tensor([[3, 2, 3, 2, 3, 2, 3, 2, 1]])
Decoded Text: 


In [ ]:
import shutil

# Create a ZIP file of your project directory
shutil.make_archive('sinhala_grammar_project', 'zip', './')


'/content/sinhala_grammar_project.zip'

In [ ]:
from google.colab import files

# Download the ZIP file
files.download('sinhala_grammar_project.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>